<div id="TOP" />
<a href="#END" > END of Notebook (filter) </a>

In [1]:
cd ~/nb/2022-Terraform/content/tf-labs/4d.Kubernetes/

# -- CONFIG ------------------------------------
LAB_TAG=${PWD##*/} 
LAB_NAME="TFI_Azure_Lab4d.Kubernetes"
LAB_WEIGHT=46
LAB_MODE="Terraform"
export MAX_LINE_LEN=80
# ----------------------------------------------

DIE() { echo "[$PWD] Error: $*"; banner "ERROR"; exit 1; }
NBTOOL_RC=~/scripts/nbtool.rc; [ ! -f $NBTOOL_RC ] && DIE "No such script as $NBTOOL_RC"
echo "==== Reading nbtool functions from '$NBTOOL_RC' ===="; . $NBTOOL_RC
EXCL_FN_INIT_NOTEBOOK
## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

bash: cd: /home/student/nb/2022-Terraform/content/tf-labs/4d.Kubernetes/: No such file or directory
==== Reading nbtool functions from '/home/student/scripts/nbtool.rc' ====
TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.

******** Mon 09 Jan 2023 06:02:14 PM UTC

## --INCLUDE--SECTION--
## --EXCLUDE--SECTION--

## TODO: Install microk8s - rerun this notebook

NB_DIR=$PWD
rm -rf ~/tmp/terraform*
#rm -rf ~/labs

FRONTMATTER=$(cat <<END_HEREDOC
---
title: 'Lab 4d.Kubernetes'
date: 2019-02-11T19:27:37+10:00
weight: 44
---

END_HEREDOC
)

echo; echo "==== Reading nbtool functions ===="
. ~/bin/nbtool.rc

echo; echo "==== Setting Exercise tag ===="
LABTAG=${PWD##*/}  # LABTAG="4d.Kubernetes"
echo "LABTAG=$LAB_TAG" | EXCL_FN_HIGHLIGHT_EOL =

echo; echo "==== Setting TF_DATA_DIR ===="
export TF_DATA_DIR=~/dot.terraform; echo "TF_DATA_DIR=$TF_DATA_DIR" | EXCL_FN_HIGHLIGHT_EOL =

NO_EXEC echo SHOULD NOT BE SEEN | EXCL_FN_HIGHLIGHT_SOL SHOULD
EXEC    echo SHOULD BE SEEN     | EXCL_FN_HIGHLIGHT_SOL SHOULD

# echo; # echo "==== Filtering current notebook ===="; EXCL_FN_FILTER_NOTEBOOK index.ipynb

echo; echo "==== Setting AWS vars for student20 account ===="
. ~/.aws/linked/student20.rc; env | grep MYAWS_ACCOUNT_NAME= | EXCL_FN_HIGHLIGHT_EOL =

# --INCLUDE--SECTION--

In [2]:
EXCL_FN_LAB_ENV

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

## Background:

Here, we will investigate the use of another Provider - the Kubernetes Provider

You can find information about the Provider here: https://registry.terraform.io/providers/hashicorp/kubernetes/latest/docs

A shared cluster has already been created on the Bastion host.

## Tasks:
### 0. Install MicroK8S - if needed
### 1. Make a directory called ‘lab4d’ underneath the labs directory.
### 2. Change into the directory.
### 3. Create the following files: main.tf, namespace.tf

# 4d.1 Verify that the Kubernetes cluster is accessible

Verify that you can use kubectl to access the cluster by using the **kubectl get nodes** command.

You should see output of the form:

# Installing Microk8s

If you do not already have a Kubernetes installation, a quick easy way to install Kubernetes on Ubuntu is to install microk8s

### Install Microk8s

```sudo snap install microk8s --classic --channel=1.26```

### Check that you have a running node

```sudo microk8s kubectl get no```

you should see output equivalent to this:
```
NAME   STATUS   ROLES    AGE     VERSION
tf     Ready    <none>   7m37s   v1.26.0
```

### Enable use of microk8s as your normal user

Join the microk8s group:

```sudo usermod -a -G microk8s student```

### Log out and log in again

You must perform this step to start a new shell with microk8s group membership

In [3]:
microk8s kubectl get nodes

NAME   STATUS   ROLES    AGE   VERSION
tf     Ready    <none>   10m   v1.26.0


### Create an alias for kubectl

To avoid having to use the microk8s command, copy the microk8s config, to ```~/.kube/config``` if that **file doesn't exist already !!**


In [24]:
# alias kubectl='microk8s kubectl'

mkdir -p ~/.kube

[ ! -f ~/.kube/config ] && microk8s config > ~/.kube/config

: 1

In [25]:
kubectl get nodes

NAME   STATUS   ROLES    AGE   VERSION
tf     Ready    <none>   27m   v1.26.0


In [4]:
#EXCLUDE
#$ kubectl get nodes

# 4d.2 Create some "kubernetes" resources

## 4d.2.1. Create the "*namespace.tf* config file

To prevent conflicts on our cluster we will first create a namespace dedicated to you

Create the namespace.tf as below, replacing *student20* by **your student user name**:

In [29]:
#EXCLUDE

cat > main.tf <<EOF

provider "kubernetes" {
  config_path    = "~/.kube/config"
  config_context = "microk8s"
}

variable "user" {
    description = "To prevent conflicts please put your student user as the default value"
    default = "student"
}

locals {
    namespace = "\${var.user}-namespace"
}

resource "kubernetes_namespace" "example" {
  metadata {
    name = local.namespace
  }
}
EOF

Create a main..tf config file as below:

In [30]:
cat main.tf


provider "kubernetes" {
  config_path    = "~/.kube/config"
  config_context = "microk8s"
}

variable "user" {
    description = "To prevent conflicts please put your student user as the default value"
    default = "student"
}

locals {
    namespace = "${var.user}-namespace"
}

resource "kubernetes_namespace" "example" {
  metadata {
    name = local.namespace
  }
}


Note how we have created a *local* variable for the namespace taking into account the **user** value.

## 4d.2.2. Intialize and then apply this config

After applying this config you should see that your student's namespace has been generated, e.g.

In [31]:
#EXCLUDE
terraform init


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/kubernetes from the dependency lock file
- Using previously-installed hashicorp/kubernetes v2.16.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [33]:
#EXCLUDE
terraform apply -auto-approve


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # kubernetes_namespace.example will be created
  + resource "kubernetes_namespace" "example" {
      + id = (known after apply)

      + metadata {
          + generation       = (known after apply)
          + name             = "student-namespace"
          + resource_version = (known after apply)
          + uid              = (known after apply)
        }
    }

Plan: 1 to add, 0 to change, 0 to destroy.
kubernetes_namespace.example: Creating...
kubernetes_namespace.example: Creation complete after 0s [id=student-namespace]

Apply complete! Resources: 1 added, 0 changed, 0 destroyed.



In [34]:
kubectl get ns


NAME                STATUS   AGE
kube-system         Active   29m
kube-public         Active   29m
kube-node-lease     Active   29m
default             Active   29m
student-namespace   Active   3s


## 4d.2.3. Create the "*deployment.tf* config file

We will now create a Kubernetes deployment.

Create the file *deployment.tf* as below, or modify as you wish - but do use the **local.namespace** which we declared earlier

In [35]:
#EXCLUDE

cat > deploy.tf <<EOF
resource "kubernetes_deployment" "k8s-demo" {
  metadata {
    name = "terraform-k8s-demo"
    namespace = local.namespace
    labels = {
      test = "k8s-demo"
    }
  }

  spec {
    replicas = 3

    selector {
      match_labels = {
        test = "k8s-demo"
      }
    }

    template {
      metadata {
        labels = {
          test = "k8s-demo"
        }
      }

      spec {
        container {
          image = "mjbright/k8s-demo:1"
          name  = "k8s-demo"

          liveness_probe {
            http_get {
              path = "/1"
              port = 80

              http_header {
                name  = "X-Custom-Header"
                value = "Ignored"
              }
            }

            initial_delay_seconds = 3
            period_seconds        = 3
          }
        }
      }
    }
  }
}
EOF

In [36]:
cat deploy.tf 


resource "kubernetes_deployment" "k8s-demo" {
  metadata {
    name = "terraform-k8s-demo"
    namespace = local.namespace
    labels = {
      test = "k8s-demo"
    }
  }

  spec {
    replicas = 3

    selector {
      match_labels = {
        test = "k8s-demo"
      }
    }

    template {
      metadata {
        labels = {
          test = "k8s-demo"
        }
      }

      spec {
        container {
          image = "mjbright/k8s-demo:1"
          name  = "k8s-demo"

          liveness_probe {
            http_get {
              path = "/1"
              port = 80

              http_header {
                name  = "X-Custom-Header"
                value = "Ignored"
              }
            }

            initial_delay_seconds = 3
            period_seconds        = 3
          }
        }
      }
    }
  }
}


## 4d.2.4. Apply this config

After applying this config you should see that your student's namespace has some resources in it

In [37]:
#EXCLUDE
TF_APPLY

kubernetes_namespace.example: Refreshing state... [id=student-namespace]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # kubernetes_deployment.k8s-demo will be created
  + resource "kubernetes_deployment" "k8s-demo" {
      + id               = (known after apply)
      + wait_for_rollout = true

      + metadata {
          + generation       = (known after apply)
          + labels           = {
              + "test" = "k8s-demo"
            }
          + name             = "terraform-k8s-demo"
          + namespace        = "student-namespace"
          + resource_version = (known after apply)
          + uid              = (known after apply)
        }

      + spec {
          + min_ready_seconds         = 0
          + paused                    = false
          + progress_deadline_seconds = 600
          + replicas           

                          + read_only   = (known after apply)
                          + secret_file = (known after apply)
                          + user        = (known after apply)

                          + secret_ref {
                              + name      = (known after apply)
                              + namespace = (known after apply)
                            }
                        }

                      + cinder {
                          + fs_type   = (known after apply)
                          + read_only = (known after apply)
                          + volume_id = (known after apply)
                        }

                      + config_map {
                          + default_mode = (known after apply)
                          + name         = (known after apply)
                          + optional     = (known after apply)

                          + items {
                              + key  = (known after apply)
                         


                          + sources {
                              + config_map {
                                  + name     = (known after apply)
                                  + optional = (known after apply)

                                  + items {
                                      + key  = (known after apply)
                                      + mode = (known after apply)
                                      + path = (known after apply)
                                    }
                                }

                              + downward_api {
                                  + items {
                                      + mode = (known after apply)
                                      + path = (known after apply)

                                      + field_ref {
                                          + api_version = (known after apply)
                                          + field_path  = (known after apply)
                             

In [39]:
kubectl get -n student-namespace all

NAME                                      READY   STATUS    RESTARTS   AGE
pod/terraform-k8s-demo-5df66b7c98-d7xn4   1/1     Running   0          25s
pod/terraform-k8s-demo-5df66b7c98-5wblp   1/1     Running   0          25s
pod/terraform-k8s-demo-5df66b7c98-r82fg   1/1     Running   0          25s

NAME                                 READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/terraform-k8s-demo   3/3     3            3           25s

NAME                                            DESIRED   CURRENT   READY   AGE
replicaset.apps/terraform-k8s-demo-5df66b7c98   3         3         3       25s


You have successfully created a namespace containing a deployment

## 4d.3 Experiment

Now experiment with this configuration

Refer to https://registry.terraform.io/providers/hashicorp/kubernetes/latest/docs, select "*Resources*" in the left-hand menu to investigate supported Kubernetes resource types.

Experiment, for example:

1. Verify that you can curl to the Pods
2. Create a file *service.tf* and define a Service for your deployment
3. Verify that you can curl to the Service
4. Modify the container image and re-apply, observe the rolling upgrade
5. Experiment with other Kubernetes resources

### 4d.4 Cleanup

In [41]:
TF_DESTROY

kubernetes_namespace.example: Refreshing state... [id=student-namespace]
kubernetes_deployment.k8s-demo: Refreshing state... [id=student-namespace/terraform-k8s-demo]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # kubernetes_deployment.k8s-demo will be destroyed
  - resource "kubernetes_deployment" "k8s-demo" {
      - id               = "student-namespace/terraform-k8s-demo" -> null
      - wait_for_rollout = true -> null

      - metadata {
          - annotations      = {} -> null
          - generation       = 1 -> null
          - labels           = {
              - "test" = "k8s-demo"
            } -> null
          - name             = "terraform-k8s-demo" -> null
          - namespace        = "student-namespace" -> null
          - resource_version = "2915" -> null
          - uid              = "f4217ea3-16ee-47c8-a181-e

          - labels           = {} -> null
          - name             = "student-namespace" -> null
          - resource_version = "2662" -> null
          - uid              = "26e4cf09-406d-4500-8c85-51ec041301d5" -> null
        }
    }

Plan: 0 to add, 0 to change, 2 to destroy.
kubernetes_namespace.example: Destroying... [id=student-namespace]
kubernetes_deployment.k8s-demo: Destroying... [id=student-namespace/terraform-k8s-demo]
kubernetes_deployment.k8s-demo: Destruction complete after 0s
kubernetes_namespace.example: Destruction complete after 6s

Destroy complete! Resources: 2 destroyed.



To destroy the Pods

<hr/>

# Summary

In this Exercise we looked at the use of another Provider - Kubernetes

We then
1. Created a Namespace
2. Created a Deployment

In [42]:
EXCL_FN_LAB_ENV

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg) -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />

# Pragma --EXCLUDE--SECTION--

## **NOTE:** Save notebook first

<div id="END" /> <a href="#TOP" > TOP OF PAGE </a>  <!-- must appear after EXCLUDE pragma !! -->

In [43]:
set +x
. ~/scripts/nbtool.rc
pwd
#NB_DIR=$PWD
#echo NB=$NB
#echo NB_DIR=$NB_DIR
#set -x
set +x
EXCL_FN_FILTER_NOTEBOOK $NB

TF-1.3.7 -rw-rw-r-- 1 student student 9 Jan  8 19:18 /home/student/.environment
Reading package lists... 0%Reading package lists... 100%Reading package lists... Done
Building dependency tree... 0%Building dependency tree... 0%Building dependency tree... 50%Building dependency tree... 50%Building dependency tree       
Reading state information... 0%Reading state information... 0%Reading state information... Done
sysvbanner is already the newest version (1:1.0-17fakesync1).
The following packages were automatically installed and are no longer required:
  linux-headers-5.4.0-120 linux-headers-5.4.0-120-generic
  linux-image-5.4.0-120-generic linux-modules-5.4.0-120-generic
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/home/student/src/mjbright.tf-scenarios-private/ServeUpLabs/content/tf-labs/4d.Kubernetes

==== Filtering current notebook [/home/student/src/mjbright.tf-scenarios-private/ServeUpLabs/content/tf-labs/4d.Kubernetes/R

In [7]:
cd $NB_DIR
nbtool.py -f index.ipynb
ls -altr *.ipynb

jupyter-nbconvert --to markdown index.ipynb.filtered.ipynb

echo "$FRONTMATTER"          > index.md
cat index.ipynb.filtered.md >> index.md
mv  index.ipynb.filtered.md index.ipynb.filtered.md.bak
ls -altr *.md

cells to include[#13]=[[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]
cells to include[#13]=[[15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 3, 2]]
-rw-rw-r-- 1 student docker  1501 Jan 31  2021 bastion.microk8s.install.ipynb
-rw-rw-r-- 1 student docker 12800 Oct  8 16:36 index.ipynb
-rw-r--r-- 1 student docker  7318 Oct  8 16:36 index.ipynb.filtered.ipynb
[NbConvertApp] Converting notebook index.ipynb.filtered.ipynb to markdown
[NbConvertApp] Writing 4807 bytes to index.ipynb.filtered.md
-rw-r--r-- 1 student docker  837 Dec 11  2020 bastion.microk8s.install.md
-rw-rw-r-- 1 student docker   79 Feb  2  2021 HEADER.md
-rw-r--r-- 1 student docker 4889 Oct  8 16:36 index.md
